In [ ]:
%%HTML
<!-- Mejorar visualización en proyector -->
<style>
.rendered_html {font-size: 1.2em; line-height: 150%;}
div.prompt {min-width: 0ex; padding: 0px;}
.container {width:95% !important;}
</style>

In [ ]:
%autosave 0
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
import torch

- https://arxiv.org/abs/1811.12560
- https://pathmind.com/wiki/deep-reinforcement-learning    
- https://towardsdatascience.com/why-going-from-implementing-q-learning-to-deep-q-learning-can-be-difficult-36e7ea1648af   

- https://github.com/deepmind/pysc2
- http://chris-chris.ai/2017/08/30/pysc2-tutorial1/
- https://github.com/chris-chris/pysc2-examples
- https://blog.goodaudience.com/lessons-and-mistakes-from-my-first-reinforcement-learning-starcraft-agent-4245cc35e956
- https://faculty.utrgv.edu/dongchul.kim/csci4352/spring2019/report/R5.pdf
- http://courses.cecs.anu.edu.au/courses/CSPROJECTS/19S1/reports/u6049249_report.pdf
- https://chatbotslife.com/building-a-smart-pysc2-agent-cdc269cb095d

- https://cs.stanford.edu/people/karpathy/reinforcejs/

## Deep Reinforced Learning

Deep Neural Networks's power to represent the world

The ability to act on that understanding

In [ ]:
Deep Learning: Takes samples from data, encode the representation in a way we can reason about it

Sequential decision making problem

In [ ]:
An inteligenet agent has to make decisions that affect the world

In [ ]:
How do we learn anything? Starting from so little?

In [ ]:
Using the frameworks of DL. The neural net is doign the representation of the world on which the actions are made